In [21]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
from openai import OpenAI
import resend
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict  # Import TypedDict for type hinting
from agents import function_tool
from agents import SQLiteSession
from IPython.display import Markdown, display


In [12]:
load_dotenv()
session = SQLiteSession("interview.db")

In [13]:
class TaviliSearchResult(TypedDict):
    query: str
    max_results: int

In [15]:
@function_tool
def tavily_search(params: TaviliSearchResult) -> str:
    """
    Calls the Tavily API and returns a string summary of top search results.

    Args:
        params (TavilySearchParams): Dictionary with 'query' (str) and 'max_results' (int).

    Returns:
        str: A formatted string summarizing the top search results, or an error message.
    """

    # The web address (endpoint) for sending search requests to Tavily ( Tavily API endpoint)
    url = "https://api.tavily.com/search" 

    # Tell the API that we’re sending JSON data
    headers = {"Content-Type": "application/json"}  

    # What we’re sending to the API:
    # Our secret API key (so Tavily knows it's us)
    # The search text (query)
    # How many results we want (defaults to 2 if not given)
    payload = {
        "api_key": tavily_api_key,  # Use the Tavily API key from environment
        "query": params["query"],   # The search query from params
        "max_results": params.get("max_results", 2),  # Use max_results from params, default to 2 if not provided
    }

    # Send the search request to Tavily (POST means we’re sending data)
    response = requests.post(url, json = payload, headers = headers) 

    # Check if the search worked (200 = OK)
    if response.status_code == 200:  # If the request was successful
        results = response.json().get("results", [])  # Extract the 'results' list from the response JSON
        
        # Build a summary string with each result's title and content, numbered
        summary = "\n".join([f"{i+1}. {r['title']}: {r['content']}" for i, r in enumerate(results)])
        return summary if summary else "No relevant results found."  # Return summary or fallback message
    else:
        return f"Tavily API error: {response.status_code}"  # Return error message with status code if request failed


In [27]:
job_search_agent = Agent(name = "Job Search Agent",
                             instructions = """
CONTEXT:
You are a world-class job research assistant with access to real-time web search via the tavily_search tool.

INSTRUCTION:
- Analyze the user's given job posting url  and determine job details, requirements, and responsibilities.
- Summarize the job posting in a clear and concise manner.
- Provide insights about the company, role, and application process.
- Provide tips for tailoring resumes and cover letters.
- Always start your answer with: "🔍 According to a job posting …"

INPUT:
You will receive a conversation history and the latest user question. Use the full context to inform your response.

OUTPUT:
Provide a clear, well-structured answer that references the search results when appropriate. If you use tavily_search, integrate the findings into your summary.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search])

print("✅ Agent created with Tavily tool.")

✅ Agent created with Tavily tool.


In [28]:
from agents import Runner

# First question
q1 = "https://www.delaware.pro/en-be/careers/jobs/junior-erp-software-developer?utm_source=Indeed&utm_medium=paid&utm_campaign=Indeed"

display(Markdown(f"**User:** {q1}"))

run1 = await Runner.run(
    starting_agent = job_search_agent,
    input = q1,
    session = session,
)

display(Markdown(f"### 🤖 Agent’s Answer\n{run1.final_output}"))

**User:** https://www.delaware.pro/en-be/careers/jobs/junior-erp-software-developer?utm_source=Indeed&utm_medium=paid&utm_campaign=Indeed

### 🤖 Agent’s Answer
🔍 According to a job posting for the Junior ERP Software Developer position at Delaware:

Job Details:
- Position: Junior ERP Software Developer
- Location: Likely Belgium (based on the URL and context)
- Role: Support development and maintenance of ERP software solutions within client projects.
- Focus: Developing ERP modules, working closely with cross-functional teams to understand client requirements, coding, testing, and troubleshooting software components.
- Entry-level position suitable for candidates starting their career in ERP software development.

Requirements:
- Basic understanding of software programming, potentially including languages like C#, .NET, or JavaScript.
- Interest or some knowledge in Microsoft Dynamics 365 or ERP systems is a plus.
- Analytical mindset with problem-solving skills.
- Team player with a willingness to learn and grow in a tech environment.
- Relevant educational background in IT, computer science, or related fields.

Company Insights:
- Delaware is a digital technology company specializing in ERP solutions, digital transformation, and consulting.
- The company values innovation, collaboration, and customer-centric solutions.

Application Tips:
- Tailor your resume to highlight any experience or coursework related to software development, ERP, or Microsoft technologies.
- Emphasize your eagerness to learn, teamwork skills, and relevant academic projects.
- In your cover letter, explain why you are interested in ERP systems and how you align with Delaware’s values and mission.
- Mention any internships, coding projects, or certifications that demonstrate your technical aptitude.

Would you like assistance drafting a tailored resume or cover letter for this role? Or guidance on next steps in the application process?